In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
import torch
from diffusers import I2VGenXLPipeline
from diffusers.utils import export_to_gif, load_image
from PIL import Image, ImageOps

# Load the pipeline
pipeline = I2VGenXLPipeline.from_pretrained("ali-vilab/i2vgen-xl", torch_dtype=torch.float16, variant="fp16")
pipeline.enable_model_cpu_offload()

# Load and process the image to preserve aspect ratio
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg/800px-Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg"
image = load_image(image_url).convert("RGB")

# Resize image while maintaining aspect ratio, with padding to fit a larger canvas
max_resolution = (768, 1024)  # Adjust resolution for portrait mode
image = ImageOps.pad(image, max_resolution, method=Image.Resampling.LANCZOS, color=(255, 255, 255))  # White padding

# Set the animation prompt and parameters
prompt = "a woman smiles"
negative_prompt = "Distorted, discontinuous, Ugly, blurry, low resolution, motionless, static, disfigured, disconnected limbs, Ugly faces, incomplete arms"
generator = torch.manual_seed(8888)

# Generate animation frames
frames = pipeline(
    prompt=prompt,
    image=image,
    num_inference_steps=50,
    negative_prompt=negative_prompt,
    guidance_scale=9.0,
    generator=generator,
    height=1024,  # Set to match the processed imag e dimensions
    width=768
).frames[0]

# Export animation to GIF
export_to_gif(frames, "i2v_full.gif")

print("GIF saved as 'i2v_full.gif'.")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

GIF saved as 'i2v_full.gif'.
